## Анализ ошибок и успехов за период

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import os
import sys
import shutil  # упаковка в архив
import warnings  # игнорирование предупреждения о формате при импорте из excel

warnings.simplefilter("ignore")

#### Загрузка данных

In [2]:
%%time
# загрузка данных ошибок, успехов и витрин

# пути к файлам
dir_path = os.getcwd().replace("\\", "/")
error_path = '/'.join(dir_path.split('/')[:3]) + '/error'
error_file_list = os.listdir(error_path)
successful_parh = '/'.join(dir_path.split('/')[:3]) + '/succsessful'
successful_file_list = os.listdir(successful_parh)
showcase_parh = '/'.join(dir_path.split('/')[:3]) + '/showcase/'
showcase_file_list = os.listdir(showcase_parh)
result_path = '/'.join(dir_path.split('/')[:3]) + '/result/'
print('созданы пути к файлам')
# список типов записи
type_list = ['3']


созданы пути к файлам
CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time
# определение типа файлов в целевой папке по расширению первого файла в директории данных ошибок, другие расширения будет нельзя
try:
    extension = error_file_list[0].split(".")[-1]
except Exception:
    print(
        "проверьте содержимое целевой директории фалов ошибок записи - в ней что-то не так"
    )
    sys.exit()
print(f"ожидаемое расширение для ВСЕХ файлов в целевой директории: {extension}")
# создание фреймов успехов и ошибок
df_error = pd.DataFrame()
for error_file in error_file_list:
    concat_flag = 0
    error_file_path = error_path + "/" + error_file
    if extension == "xlsx" and error_file.split('.')[-1] == 'xlsx':
        try:
            df_error_day = pd.read_excel(
                error_file_path, sheet_name="1", index_col=None, dtype="str"
            )
            print(f"\t- успешно прочитан файл {error_file_path}")
            concat_flag = 1
        except Exception:
            print(f"\tошибка обработки файла {error_file}, файл пропущен")
    elif extension == "csv" and error_file.split('.')[-1] == 'csv':
        try:
            df_error_day = df_ass = pd.read_csv(
                error_file_path, sep=";", encoding="utf-8-sig", dtype="str"
            )
            print(f"\t- успешно прочитан файл {error_file_path}")
            concat_flag = 1
        except Exception:
            print(f"\tошибка обработки файла {error_file}, файл пропущен")
    else:
        print(f"\tфайл {error_file} имеет неверный тип, файл пропущен")
    if concat_flag == 1:
        df_error = pd.concat([df_error, df_error_day])
        print(f'\tданные из {error_file} присоединены к набору')
    else:
        print(f'\tданные из {error_file} не присоединены к набору')
df_error = df_error.reset_index(drop=True).rename(
    columns={
        "Название субъекта РФ": "region_name",
        "Тип записи": "appointment_type",
        "Post_name": "post_name",
    }
)
df_error["create_ts"] = pd.to_datetime(df_error["create_ts"], format="mixed")
# вставка столбца data в таблицу df_error
df_error.insert(
    0,
    "date",
    df_error["create_ts"].apply(
        lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
    ),
)
df_error["region_name"] = df_error["region_name"].fillna('Кемская волость')
df_error = df_error[df_error["appointment_type"].isin(type_list)]
print("данные ошибок записи сформированы")

df_succsessful = pd.DataFrame()
for succsessful_file in successful_file_list:
    concat_flag = 0
    succsessful_file_path = successful_parh + "/" + succsessful_file
    if extension == "xlsx" and succsessful_file.split('.')[-1] == 'xlsx':
        try:
            df_succsessful_day = pd.read_excel(
                succsessful_file_path, sheet_name="1", index_col=None, dtype="str"
            )
            print(f"\t- успешно прочитан файл {succsessful_file_path}")
            concat_flag = 1
        except Exception:
            print(f"\tошибка обработки файла {succsessful_file}, файл пропущен")
    elif extension == "csv" and succsessful_file.split('.')[-1] == 'csv':
        try:
            df_succsessful_day = df_ass = pd.read_csv(
                succsessful_file_path, sep=";", encoding="utf-8-sig", dtype="str"
            )
            print(f"\t- успешно прочитан файл {succsessful_file_path}")
            concat_flag = 1
        except Exception:
            print(f"\tошибка обработки файла {succsessful_file}, файл пропущен")
    else:
        print(f"\tфайл {succsessful_file} имеет неверный тип, файл пропущен")
    if concat_flag == 1:
        df_succsessful = pd.concat([df_succsessful, df_succsessful_day])
        print(f'\tданные из {succsessful_file} присоединены к набору')
    else:
        print(f'\tданные из {succsessful_file} не присоединены к набору')
df_succsessful = df_succsessful.reset_index(drop=True).rename(
    columns={
        "Дата": "date",
        "Название субъекта РФ": "region_name",
        "Тип записи": "appointment_type",
    }
)
df_succsessful["date"] = df_succsessful["create_ts"].apply(
    lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
)
df_succsessful["slot_ts"] = pd.to_datetime(df_succsessful["slot_ts"], format="mixed")
df_succsessful["create_ts"] = pd.to_datetime(
    df_succsessful["create_ts"], format="mixed"
)
df_succsessful["region_name"] = df_succsessful["region_name"].fillna('Кемская волость')
df_succsessful = df_succsessful[df_succsessful["appointment_type"].isin(type_list)]
print("данные успешных сессий сформированы")

# создание фрейма данных витрин
df_showcase = pd.read_excel(
    showcase_parh + showcase_file_list[-1],
    sheet_name="Сессии по датам",
    engine='openpyxl',
    index_col=None,
    dtype="str",
)
df_showcase["Дата"] = pd.to_datetime(df_showcase["Дата"], format="mixed")
df_showcase = df_showcase.rename(
    columns={
        "Код региона": "region_code",
        "Регион": "region_name",
        "Дата": "date",
        "Шаг": "step",
        "Всего": "total",
        "Успешно": "succsessful",
        "Ошибки ЕПГУ": "epgu_error",
        "Тех ошибки": "tech_error",
        "Орг ошибки": "org_error",
        "ПОДД": "podd",
        "Брошенные": "draft",
        "Бизнес-правила": "b-rules"
    }
)
df_showcase[df_showcase.columns[4:]] = df_showcase[df_showcase.columns[4:]].apply(
    pd.to_numeric, errors="ignore"
)
df_showcase["region_name"] = df_showcase["region_name"].fillna('Кемская волость')
print("данные витрин ЕПГУ загружены")

ожидаемое расширение для ВСЕХ файлов в целевой директории: csv
	- успешно прочитан файл c:/work/comparison_error_successful/error/01.12.2023.csv
	данные из 01.12.2023.csv присоединены к набору
	- успешно прочитан файл c:/work/comparison_error_successful/error/02.12.2023.csv
	данные из 02.12.2023.csv присоединены к набору
	- успешно прочитан файл c:/work/comparison_error_successful/error/03.12.2023.csv
	данные из 03.12.2023.csv присоединены к набору
	- успешно прочитан файл c:/work/comparison_error_successful/error/04.12.2023.csv
	данные из 04.12.2023.csv присоединены к набору
	- успешно прочитан файл c:/work/comparison_error_successful/error/05.12.2023.csv
	данные из 05.12.2023.csv присоединены к набору
	- успешно прочитан файл c:/work/comparison_error_successful/error/06.12.2023.csv
	данные из 06.12.2023.csv присоединены к набору
	- успешно прочитан файл c:/work/comparison_error_successful/error/07.12.2023.csv
	данные из 07.12.2023.csv присоединены к набору
	- успешно прочитан файл c:

#### Однообразный диапазон дат

In [4]:
date_begin = max(df_error.date.min(), df_succsessful.date.min(), df_showcase.date.min())
date_end = min(df_error.date.max(), df_succsessful.date.max(), df_showcase.date.max())
df_error = df_error[
    (df_error.date >= date_begin) & (df_error.date <= date_end)
].reset_index(drop=True)
df_succsessful = df_succsessful[
    (df_succsessful.date >= date_begin) & (df_succsessful.date <= date_end)
].reset_index(drop=True)
df_showcase = df_showcase[
    (df_showcase.date >= date_begin) & (df_showcase.date <= date_end)
].reset_index(drop=True)
print(f"В отчёте даты с {date_begin} по {date_end}")

В отчёте даты с 2023-12-01 00:00:00 по 2023-12-31 00:00:00


#### Однообразный перечень регионов
###### `Серёжа, а ты не забыл актуализировать таблицу соответствия названий регионов в директории исполняемого файла скрипта?`

In [5]:
# загрузка справочника, который хранится в директории скрипта
df_reg_ref = pd.read_csv(dir_path + "/" + "reg_ref.csv", sep=";")

# присоединение единого названия региона
df_error = df_error.merge(
    df_reg_ref, left_on="region_name", right_on="region_name_file", how="left"
)
df_error = df_error.drop("region_name_file", axis=1)

df_succsessful = df_succsessful.merge(
    df_reg_ref, left_on="region_name", right_on="region_name_file", how="left"
)
df_succsessful = df_succsessful.drop("region_name_file", axis=1)

df_showcase = df_showcase.merge(
    df_reg_ref, left_on="region_name", right_on="region_name_file", how="left"
)
df_showcase = df_showcase.drop("region_name_file", axis=1)

##### Проверки на дубли в session_id

In [17]:
dup_succsessful = df_succsessful.loc[
    df_succsessful.duplicated(subset=["date", "session_id"], keep=False)
].sort_values(by=["date", "session_id"])
if dup_succsessful.shape[0] / 2 == len(dup_succsessful["session_id"].unique()):
    print(
        f"в успехах {len(dup_succsessful['session_id'].unique())} повторяющихся session_id внутри одной даты"
    )
elif dup_succsessful.shape[0] / 2 != len(dup_succsessful["session_id"].unique()):
    print(f"в ошибках какая-то фигня с повторами в session_id в успехах")
else:
    print("рандомный трабл")

dup_error = df_error.loc[
    df_error.duplicated(subset=["date", "session_id"], keep=False)
].sort_values(by=["date", "session_id"])
if dup_error.shape[0] / 2 == len(dup_error["session_id"].unique()):
    print(
        f"в ошибках {len(dup_error['session_id'].unique())} повторяющихся session_id внутри одной даты"
    )
elif dup_error.shape[0] / 2 != len(dup_error["session_id"].unique()):
    print(f"в ошибках какая-то фигня с повторами в session_id в ошибках")
else:
    print("рандомный трабл")

в успехах 0 повторяющихся session_id внутри одной даты
в ошибках 43 повторяющихся session_id внутри одной даты


#### Статистика ошибок записи через ЕПГУ

##### a) cчитаем количество уникальных session_id в разрезе final_state

In [22]:
# группировка по дате, региогну и событию
df_error_group = (
    df_error.groupby(["date", "real_region_name", "final_state"])["session_id"]
    .nunique()
    .reset_index()
    .rename(columns={"final_state": "event", "session_id": "value"})
)
# перечень всех событий
df_error_event = (
    pd.DataFrame(df_error_group.event)
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={"event": "event"})
)
# перечни регионов и дат
date_list = df_error.date.unique().tolist()
region_list = df_error.real_region_name.unique().tolist()
# формирование набора статистических данных для ошибок записи
df_error_statistic = pd.DataFrame()
for date in date_list:
    for region in region_list:
        df_loc_day = df_error_group[
            (df_error_group.real_region_name == region) & (df_error_group.date == date)
        ]
        df_loc_day = df_loc_day.merge(df_error_event, on="event", how="right")
        df_loc_day["date"] = df_loc_day["date"].fillna(date)
        df_loc_day["real_region_name"] = df_loc_day["real_region_name"].fillna(region)
        df_loc_day["value"] = df_loc_day["value"].fillna(0)
        df_error_statistic = pd.concat([df_error_statistic, df_loc_day])

##### б) cчитаем количество строк в разрезе final_state

In [19]:
# группировка по дате, региогну и событию
df_error_group = (
    df_error.groupby(["date", "real_region_name", "final_state"])["session_id"]
    .count()
    .reset_index()
    .rename(columns={"final_state": "event", "session_id": "value"})
)
# перечень всех событий
df_error_event = (
    pd.DataFrame(df_error_group.event)
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={"event": "event"})
)
# перечни регионов и дат
date_list = df_error.date.unique().tolist()
region_list = df_error.real_region_name.unique().tolist()
# формирование набора статистических данных для ошибок записи
df_error_statistic = pd.DataFrame()
for date in date_list:
    for region in region_list:
        df_loc_day = df_error_group[
            (df_error_group.real_region_name == region) & (df_error_group.date == date)
        ]
        df_loc_day = df_loc_day.merge(df_error_event, on="event", how="right")
        df_loc_day["date"] = df_loc_day["date"].fillna(date)
        df_loc_day["real_region_name"] = df_loc_day["real_region_name"].fillna(region)
        df_loc_day["value"] = df_loc_day["value"].fillna(0)
        df_error_statistic = pd.concat([df_error_statistic, df_loc_day])

#### Статистика успешных записей через ЕПГУ

##### а) cчитаем количество уникальных session_id

In [36]:
df_succsessful_statistic = (
    df_succsessful.groupby(["date", "real_region_name"])["session_id"]
    .nunique()
    .reset_index()
    .rename(columns={"session_id": "value"})
)
df_succsessful_statistic.insert(2, "event", "Успешная запись (ЕПГУ)")

##### б) cчитаем количество строк

In [20]:
df_succsessful_statistic = (
    df_succsessful.groupby(["date", "real_region_name"])["session_id"]
    .count()
    .reset_index()
    .rename(columns={"session_id": "value"})
)
df_succsessful_statistic.insert(2, "event", "Успешная запись (ЕПГУ)")

#### Статистика использования витрин

In [22]:
# успешные записи через витрины
df_showcase_succsessuil = df_showcase[
    df_showcase.step == "Шаг 4. Запись. Уникальные сессии"
][["date", "real_region_name", "succsessful"]].rename(columns={"succsessful": "value"})
df_showcase_succsessuil.insert(2, "event", "Успешные записи (витрины)")
# количество ошибок через витрины
df_showcase_error = (
    df_showcase.groupby(["date", "real_region_name"])[
        ["epgu_error", "tech_error", "org_error", "podd", "draft", "b-rules"]
    ]
    .sum()
    .reset_index()
)
# ошибки записи через витрины
df_showcase_error.rename(
    columns={
        "epgu_error": "Ошибки ЕПГУ (витрины)",
        "tech_error": "Технические ошибки (витрины)",
        "org_error": "Организационные ошибки (витрины)",
        "podd": "ПОДД (витрины)",
        "draft": "Брошеные (витрины)",
        "b-rules": "Бизнес-правила (витрины)",
    },
    inplace=True,
)
# преобразование в "длинную" таблицу
df_showcase_error = pd.melt(
    df_showcase_error,
    id_vars=["date", "real_region_name"],
    value_vars=[
        "Ошибки ЕПГУ (витрины)",
        "Технические ошибки (витрины)",
        "Организационные ошибки (витрины)",
        "ПОДД (витрины)",
        "Брошеные (витрины)",
        "Бизнес-правила (витрины)",
    ],
)
df_showcase_error.rename(columns={"variable": "event"}, inplace=True)
df_showcase_statistic = (
    pd.concat([df_showcase_succsessuil, df_showcase_error])
    .sort_values(by=["date", "real_region_name", "event"])
    .reset_index(drop=True)
)

#### Профиль региона

In [23]:
# объединение статистики успехов, ошибок и витрин
df_statistic = pd.concat([df_error_statistic, df_succsessful_statistic]).sort_values(
    by=["date", "real_region_name"]
)

df_statistic = (
    pd.concat([df_statistic, df_showcase_statistic])
    .sort_values(by=["date", "real_region_name"])
    .reset_index(drop=True)
)

# удаление строк спустыми регионами если для них все предикторы равны 0
if df_statistic[df_statistic["real_region_name"].isna()]["value"].sum() < 0.1:
    df_statistic.dropna(subset=["real_region_name"], inplace=True)
# предикторы профиля
df_full_event = (
    pd.DataFrame(df_statistic["event"]).drop_duplicates().reset_index(drop=True)
)
print(f"Всего в профиле региона {df_full_event.shape[0]} предиктора(ов)")

# создание профиля региона
df_stat_full = pd.DataFrame()
for date in df_statistic["date"].unique().tolist():
    for region in df_statistic["real_region_name"].unique().tolist():
        df_stat_tmp = df_statistic[
            (df_statistic["date"] == date)
            & (df_statistic["real_region_name"] == region)
        ]
        df_stat_tmp = df_stat_tmp.merge(
            df_full_event, left_on="event", right_on="event", how="right"
        )
        df_stat_tmp["date"].fillna(date, inplace=True)
        df_stat_tmp["real_region_name"].fillna(region, inplace=True)
        df_stat_tmp["value"].fillna(0, inplace=True)
        df_stat_full = pd.concat([df_stat_full, df_stat_tmp])
        # break
df_stat_full.reset_index(drop=True, inplace=True)
df_stat_full["value"] = df_stat_full["value"].astype(int)

Всего в профиле региона 24 предиктора(ов)


In [28]:
# df_stat_full[
#     (df_stat_full.real_region_name == "Самарская область")
#     & (df_stat_full.date == "2024-01-10")
# ]
# df_stat_full.head(df_full_event.shape[0])
df_stat_full

,date,real_region_name,event,value
0,2023-12-01,Алтайский край,Внутренняя ошибка РМИС при создании записи,82
1,2023-12-01,Алтайский край,Значение не найдено,113
2,2023-12-01,Алтайский край,Ошибка «РМИС не ответил»,53
3,2023-12-01,Алтайский край,Ошибка «СМЭВ-timeout»,2
4,2023-12-01,Алтайский край,Ошибка валидации данных пациента,97
...,...,...,...,...
65467,2023-12-31,г. Севастополь,Организационные ошибки (витрины),0
65468,2023-12-31,г. Севастополь,Ошибки ЕПГУ (витрины),0
65469,2023-12-31,г. Севастополь,ПОДД (витрины),0
65470,2023-12-31,г. Севастополь,Технические ошибки (витрины),0


#### Сравнение данных

In [29]:
%%time
# вытаскивание таблички "Сверка" из файлов чат-бота
example_path = "/".join(dir_path.split("/")[:3]) + "/bots/"
example_file_list = os.listdir(example_path)
df_bot = pd.DataFrame()
for file_name in example_file_list:
    df_tmp = pd.read_excel(
        example_path + file_name,
        sheet_name="Сверка",
        engine="openpyxl",
        index_col=None,
    ).rename(
        columns={
            "Тех ошибки": "Всего тех",
            "Unnamed: 4": "Новые тех",
            "Unnamed: 5": "Тех без новых",
        }
    )
    df_tmp.drop(index=[0], inplace=True)
    df_tmp.drop(index=[x for x in range(5, df_tmp.shape[0] + 1)], inplace=True)
    df_tmp.insert(0, "date", file_name.split(".")[0].split("_")[-1])
    df_bot = pd.concat([df_bot, df_tmp])
    print(f"загружено {file_name}, полёт нормальный")
    # break
df_bot.reset_index(drop=True, inplace=True)

загружено ЭЗ_бот_2023-12-01.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-02.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-03.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-04.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-05.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-06.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-07.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-08.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-09.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-10.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-11.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-12.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-13.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-14.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-15.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-16.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-17.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-18.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-19.xlsm, полёт нормальный
загружено ЭЗ_бот_2023-12-20.xls

In [30]:
df_bot

,date,КУ ФЭР,Успешные,Орг ошибки,Всего тех,Новые тех,Тех без новых,Ошибки ФЭР
0,2023-12-01,РФ,124209.0,40271.0,91018,1374,89644,19759.0
1,2023-12-01,г. Байконур,0.0,0.0,0,0,0,0.0
2,2023-12-01,Федеральное медико-биологическое агентство,0.0,0.0,0,0,0,0.0
3,2023-12-01,РФ без Байконура и ФМБА,124209.0,40271.0,91018,1374,89644,19759.0
4,2023-12-02,РФ,57041.0,20647.0,18568,732,17836,9385.0
...,...,...,...,...,...,...,...,...
119,2023-12-30,РФ без Байконура и ФМБА,26589.0,6318.0,7972,311,7661,4167.0
120,2023-12-31,РФ,12200.0,3520.0,4442,66,4376,2117.0
121,2023-12-31,г. Байконур,0.0,0.0,0,0,0,0.0
122,2023-12-31,Федеральное медико-биологическое агентство,3.0,0.0,0,0,0,0.0


______________________________________

In [20]:
for date in date_list:
    df_showcase_test = df_showcase[df_showcase['date'] == date]
    break
df_showcase_test.head(30)

,region_code,region_name,date,step,total,succsessful,epgu_error,tech_error,org_error,podd,draft,Бизнес-правила
0,30,Астраханская область,2024-01-01,Шаг 1. Идентификация пациента в регионе. Уника...,1335,1327,0,1,7,0,0,0.0
1,30,Астраханская область,2024-01-01,Шаг 2. Выбор специальности. Уникальные сессии,1327,1221,0,47,4,0,55,0.0
2,30,Астраханская область,2024-01-01,Шаг 3. Выбор врача и времени. Уникальные сессии,1221,1091,0,1,58,0,71,0.0
3,30,Астраханская область,2024-01-01,Шаг 4. Запись. Уникальные сессии,1091,335,0,0,0,0,736,20.0
4,32,Брянская область,2024-01-01,Шаг 1. Идентификация пациента в регионе. Уника...,3884,3866,0,1,17,0,0,0.0
5,32,Брянская область,2024-01-01,Шаг 2. Выбор специальности. Уникальные сессии,3866,3604,0,1,26,0,235,0.0
6,32,Брянская область,2024-01-01,Шаг 3. Выбор врача и времени. Уникальные сессии,3604,2648,0,0,636,0,320,0.0
7,32,Брянская область,2024-01-01,Шаг 4. Запись. Уникальные сессии,2648,551,0,43,165,0,1883,6.0
8,33,Владимирская область,2024-01-01,Шаг 1. Идентификация пациента в регионе. Уника...,3021,3008,0,0,13,0,0,0.0
9,33,Владимирская область,2024-01-01,Шаг 2. Выбор специальности. Уникальные сессии,3008,2870,0,0,4,0,134,0.0


In [21]:
# min(df_error.create_ts)
# timestamp_string = "2023-07-21 15:30:45"
# format_string = "%Y-%m-%d %H:%M:%S"
# datetime_object = datetime.strptime(str(min(df_error.create_ts))[:10], '%Y-%m-%d')
datetime.strptime(str(min(df_error.create_ts))[:10], '%Y-%m-%d').date()

datetime.date(2024, 1, 1)

### Количество пропусков

In [22]:
pd.DataFrame(df_error.isnull().sum()).reset_index().rename(columns={'index': 'col', 0: 'count'})

,col,count
0,date,0
1,session_id,0
2,region_name,3
3,step_id,0
4,step_name,0
5,error_code,0
6,final_state,0
7,mo_oid,455010
8,mo_name,445622
9,post_name,650730


In [23]:
pd.DataFrame(df_succsessful.isnull().sum()).reset_index().rename(columns={'index': 'col', 0: 'count'})

,col,count
0,date,0
1,session_id,0
2,region_name,0
3,sp_oid,7321
4,sp_name,0
5,post_name,0
6,slot_ts,0
7,create_ts,0
8,appointment_type,0


In [24]:
pd.DataFrame(df_showcase.isnull().sum()).reset_index().rename(columns={'index': 'col', 0: 'count'})

,col,count
0,region_code,0
1,region_name,0
2,date,0
3,step,0
4,total,0
5,succsessful,0
6,epgu_error,0
7,tech_error,0
8,org_error,0
9,podd,0


In [25]:
source_path = dir_path + "/source"
save_path = dir_path + "/error_log/"
file_list = os.listdir(source_path)

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'c:/work/comparison_error_successful/comparison/source'

In [2]:
%%time

import pandas as pd
import os

dir_path = os.getcwd()
dir_ = os.listdir(dir_path.replace("\\", "/"))
list_files = []
for element in dir_:
    try:
        if element.split(".")[-1] == "csv" and element.split(".")[0] != "result":
            list_files.append(element)
    except Exception:
        pass
    try:
        if element.split(".")[-1] == "xlsx":
            part_sh = dir_path.replace("\\", "/") + "/" + element
    except Exception:
        pass
try:
    if os.path.getsize(
        dir_path.replace("\\", "/") + "/" + list_files[0]
    ) > os.path.getsize(dir_path.replace("\\", "/") + "/" + list_files[1]):
        path_error = dir_path.replace("\\", "/") + "/" + list_files[0]
        path_suc = dir_path.replace("\\", "/") + "/" + list_files[1]
    else:
        path_error = dir_path.replace("\\", "/") + "/" + list_files[1]
        path_suc = dir_path.replace("\\", "/") + "/" + list_files[0]
except Exception:
    print("Что-то пошло не так. Будте внимательнее и попробуйте ещё раз.")

path_result = dir_path.replace("\\", "/") + "/result.csv"

df_test = pd.read_csv(path_error, sep=";", dtype=object, header=None)
# замена пустого значения в столбце региона на... непустое значение
df_test[1] = df_test[1].fillna("Регион не определён")
df_check = df_test.copy() # для опытов
df_check_0 = df_test.copy() # для опытов
error_type_list = [
    "Внутренняя ошибка РМИС при создании записи",
    "Ошибка «РМИС не ответил»",
    "Ошибка валидации данных пациента",
    "Ошибка валидации схемы",
    "Сервис МИС приостановлен",
]
error_type_list_fer = [
    "Внутренняя ошибка концентратора при создании записи",
    "Ошибка «СМЭВ-timeout»",
    "Ошибка во входных параметрах при записи на прием",
]
df_tech_error = df_test.loc[(df_test[5].isin(error_type_list)) & (df_test[11] == "3")]
df_tech_error = pd.DataFrame(df_tech_error.groupby(1)[5].count()).reset_index()
df_org_error = df_test.loc[(df_test[5] == "Значение не найдено") & (df_test[11] == "3")]
df_org_error = pd.DataFrame(df_org_error.groupby(1)[5].count()).reset_index()
df_regions = (
    pd.DataFrame(df_test[1]).drop_duplicates().drop(index=0).reset_index(drop=True)
)
df_fer_error = df_test.loc[
    (df_test[5].isin(error_type_list_fer)) & (df_test[11] == "3")
]

# df_fer_error_test = df_fer_error.copy() # для проверки ошибок ФЕР с нулевым регионом
df_fer_error = pd.DataFrame(df_fer_error.groupby(1)[5].count()).reset_index()

df_errors = (
    df_regions.merge(df_tech_error, left_on=1, right_on=1, how="left")
    .merge(df_org_error, left_on=1, right_on=1, how="left")
    .merge(df_fer_error, left_on=1, right_on=1, how="left")
)
df_errors = df_errors.rename(
    columns={
        1: "region_name",
        "5_x": "tech_errors",
        "5_y": "org_errors",
        5: "FER_errors",
    }
)
df_errors = df_errors.fillna(0)
df_errors[["tech_errors", "org_errors", "FER_errors"]] = df_errors[
    ["tech_errors", "org_errors", "FER_errors"]
].astype("int")
df_test = pd.read_csv(path_suc, sep=";", dtype=object, header=None)
df_suc_test = df_test.copy() # для опытов 2
# замена пустого значения в столбце региона на... непустое значение
df_test[2] = df_test[2].fillna("Регион не определён")

df_suc = pd.DataFrame(
    df_test.loc[df_test[8] == "3"].groupby(2).count()[0]
).reset_index()
df_full = (
    df_errors.merge(df_suc, left_on="region_name", right_on=2, how="left")
    .rename(columns={0: "success"})[
        ["region_name", "success", "org_errors", "FER_errors", "tech_errors"]
    ]
    .fillna(0)
)
df_full["success"] = df_full["success"].astype("int")

df_full["success_ratio(%)"] = round(
    df_full["success"]
    * 100
    / (df_full["success"] + df_full["org_errors"] + df_full["tech_errors"]),
    1,
).fillna(100.0)
df_full["org_errors_ratio(%)"] = round(
    df_full["org_errors"]
    * 100
    / (df_full["success"] + df_full["org_errors"] + df_full["tech_errors"]),
    1,
).fillna(0.00)
df_full["tech_errors_ratio(%)"] = round(
    df_full["tech_errors"]
    * 100
    / (df_full["success"] + df_full["org_errors"] + df_full["tech_errors"]),
    1,
).fillna(0.00)
df_full = df_full[
    [
        "region_name",
        "success",
        "success_ratio(%)",
        "org_errors",
        "org_errors_ratio(%)",
        "tech_errors",
        "tech_errors_ratio(%)",
        "FER_errors",
    ]
]

df_full = df_full.sort_values("success_ratio(%)", ascending=False).reset_index(
    drop=True
)

sum_row = {
    col: df_full[col].sum()
    for col in ["success", "org_errors", "FER_errors", "tech_errors"]
}
sum_df = pd.DataFrame(sum_row, index=["Total"])
sum_df.insert(0, "region_name", "TOTAL")
sum_df = sum_df.reset_index(drop=True)
df_full = pd.concat([df_full, sum_df], ignore_index=True).fillna("")

df_full.to_csv(path_result, sep=";", encoding="utf-8-sig", index=False)
print(
    "\n\n\t\t\tКоличество (доля) успешных записей и ошибок по субъектам (ошибки ФЭР приведены для справки, в расчёте долей не применяются)\n"
)
print(df_full.to_string(index=False))

# проверка на пустой регион
df_check = df_check.rename(columns=df_check.iloc[0]).drop(index=0)
df_check_actual = df_check[
    ((df_check["final_state"].isin(error_type_list))
    | (df_check["final_state"].isin(error_type_list_fer))
    | (df_check["final_state"] == "Значение не найдено"))
    & ((df_check["Тип записи"] == "3"))
]
df_check_nanregion = df_check_actual[df_check_actual["Название субъекта РФ"] == "Регион не определён"]
if df_check_nanregion.empty:
    print("Пустых регионов нет")
else:
    print("\n\n\t\t\tПустые регионы:\n")
    print(df_check_nanregion[
    [
        "session_id",
        "step_id",
        "step_name",
        "error_code",
        "final_state",
    ]
].to_string(index=True))
print("\n\n\t\t\tВсё\n")



			Количество (доля) успешных записей и ошибок по субъектам (ошибки ФЭР приведены для справки, в расчёте долей не применяются)

                             region_name  success success_ratio(%)  org_errors org_errors_ratio(%)  tech_errors tech_errors_ratio(%)  FER_errors
                   Волгоградская область     2949             91.6         108                 3.4          164                  5.1         482
                         Республика Тыва      830             89.4          32                 3.4           66                  7.1          56
                    Чеченская Республика     2156             89.1          66                 2.7          198                  8.2          81
                    Чувашская Республика     3262             89.1         122                 3.3          278                  7.6         542
         Карачаево-Черкесская Республика      187             86.2           7                 3.2           23                 10.6          25


In [2]:
path_error

'c:/work/new_bot/script/09.08.csv'

In [4]:
# part_sh
df_sh = pd.read_excel(
    part_sh, sheet_name="Сессии по датам", index_col=None, dtype="str"
)
df_sh = df_sh.apply(pd.to_numeric, errors="ignore")
df_sh["Дата"] = pd.to_datetime(df_sh["Дата"], format="%Y-%m-%d %H:%M:%S")
df_sh_group = df_sh.loc[df_sh['Дата'] == df_sh['Дата'].max()][[df_sh.columns[1], *list(df_sh.columns[4:])]].groupby('Регион').sum().reset_index()

In [5]:
df_sh_group

,Регион,Всего,Успешно,Ошибки ЕПГУ,Тех ошибки,Орг ошибки,ПОДД,Брошенные
0,Астраханская область,14898,11874,0,96,402,19,2507
1,Брянская область,37157,28173,1,100,2717,59,6107
2,Владимирская область,32055,24629,1,6,1717,69,5633
3,Вологодская область,18667,14178,0,12,1496,0,2981
4,Иркутская область,20757,15423,0,46,976,1,4311
5,Кабардино-Балкарская Республика,10578,8832,0,2,124,0,1620
6,Калужская область,24949,19215,0,25,1550,1,4158
7,Кировская область,24643,18840,0,1,1515,0,4287
8,Курганская область,3478,2559,0,56,247,0,616
9,Липецкая область,25932,18839,0,164,2034,579,4316


In [41]:
df_allfull = df_full.merge(df_sh_group, left_on='region_name', right_on='Регион', how='left')

In [43]:
df_allfull
print(df_allfull.to_string(index=False))

                               region_name  success success_ratio(%)  org_errors org_errors_ratio(%)  tech_errors tech_errors_ratio(%)  FER_errors                                   Регион    Всего  Успешно  Ошибки ЕПГУ  Тех ошибки  Орг ошибки  ПОДД  Брошенные
Федеральное медико-биологическое агентство        0            100.0           0                 0.0            0                  0.0           0                                      NaN      NaN      NaN          NaN         NaN         NaN   NaN        NaN
                           Республика Тыва     1199             89.9          64                 4.8           70                  5.3         179                                      NaN      NaN      NaN          NaN         NaN         NaN   NaN        NaN
                     Волгоградская область     2110             87.6         127                 5.3          173                  7.2         698                                      NaN      NaN      NaN          NaN  